<a href="https://colab.research.google.com/github/RG2806/ai_spotle/blob/master/CNN-78.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!git clone https://github.com/RG2806/ai_spotle.git

Cloning into 'ai_spotle'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 6 (delta 0), reused 3 (delta 0), pack-reused 0
Unpacking objects: 100% (6/6), done.


In [16]:
%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split

from keras.models import Sequential #Initialise our neural network model as a sequential network
from keras.layers import Conv2D #Convolution operation
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.layers import Activation#Applies activation function
from keras.layers import Dropout#Prevents overfitting by randomly converting few outputs to zero
from keras.layers import MaxPooling2D # Maxpooling function
from keras.layers import Flatten # Converting 2D arrays into a 1D linear vector
from keras.layers import Dense # Regular fully connected neural network
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint
from sklearn.metrics import accuracy_score

import pandas as pd
import keras 
from keras.models import Sequential
from keras.layers import Conv2D, BatchNormalization, GlobalAveragePooling2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras.utils import np_utils
import keras.backend as k
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.regularizers import l2


In [24]:
def load_data(dataset_path):
  
  classes = ['Fear', 'Happy', 'Sad']  #We will be dealing with seven different types of emotions.

  data = []
  labels =[]

  with open(dataset_path, 'r') as file:
      for line_no, line in enumerate(file.readlines()):
          if 0 < line_no <= 35887:
            line = line.split(',')
            image_data = np.asarray([int(x) for x in line[1:]]).reshape(48, 48)#Creating a list out of the string then converting it into a 2-Dimensional numpy array.
            image_data =image_data.astype(np.uint8)/255.0
            data.append(image_data)
            labels.append(classes.index(line[0]))
      data = np.expand_dims(data, -1)   
      #labels = to_categorical(labels, num_classes = 3)
    
      return np.array(data), np.array(labels)

In [25]:
dataset_path = "/content/ai_spotle/aithon2020_level2_traning.csv" 
train_data, train_labels = load_data(dataset_path)
#train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size = test_size,random_state = seed)

print("Number of images in Training set:", len(train_data))

Number of images in Training set: 10817


In [26]:
train_labels.shape

(10817,)

In [27]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train_data, train_labels, test_size=0.2, random_state=101)
y_train = np_utils.to_categorical(y_train, 3)
y_test = np_utils.to_categorical(y_test, 3)

In [39]:
num_features = 64
width, height = 48, 48
model = Sequential()

model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(width, height, 1), data_format='channels_last', kernel_regularizer=l2(0.01)))
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(2*2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*num_features, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(3, activation='softmax'))
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_71 (Conv2D)           (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_72 (Conv2D)           (None, 46, 46, 64)        36928     
_________________________________________________________________
batch_normalization_64 (Batc (None, 46, 46, 64)        256       
_________________________________________________________________
max_pooling2d_28 (MaxPooling (None, 23, 23, 64)        0         
_________________________________________________________________
dropout_54 (Dropout)         (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_73 (Conv2D)           (None, 23, 23, 128)       73856     
_________________________________________________________________
batch_normalization_65 (Batc (None, 23, 23, 128)      

In [30]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
              metrics=['accuracy'])

lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3, verbose=1)
early_stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=8, verbose=1, mode='auto')
checkpointer = ModelCheckpoint('../input/model.h5', monitor='val_loss', verbose=1, save_best_only=True)

callbacks = [lr_reducer, early_stopper, checkpointer]

In [ ]:
# With Augmentation
bs = 64
epochs = 100

aug = ImageDataGenerator(rotation_range=20, 
                         width_shift_range=0.1, 
                         height_shift_range=0.1, 
                         horizontal_flip=True, 
                         fill_mode="nearest")

H = model.fit_generator(aug.flow(x_train, y_train, batch_size=bs),
                        validation_data=(x_test, y_test), 
                        steps_per_epoch=len(x_train)//bs,
                        callbacks=callbacks,
                        shuffle=True,
                        epochs=epochs)

In [ ]:
scores = model.evaluate(x_test, y_test, batch_size=bs)
print("Loss: " + str(scores[0]))
print("Accuracy: " + str(scores[1]))

In [33]:
model.save_weights("/content/model.h5")
print("Saved model to disk")

Saved model to disk
